In [ ]:
#@title 1. Check current GPU assigned
!nvidia-smi -L
!nvidia-smi

In [ ]:
#@title 2. Download stable-diffusion Repository
from os.path import exists

if exists("/content/stable-diffusion/")==True:
  print("Already downloaded repo")
else:
  !git clone --quiet https://github.com/kylejmorris/stable-diffusion-textual-inversion # Original repo
  %mv stable-diffusion-textual-inversion stable-diffusion
  %cd stable-diffusion/
  !git checkout --quiet tags/release-1.09
  

In [ ]:
#@title 3. Install Python 3.8 
%%capture --no-stderr
import gc
!apt-get -qq install python3.8
gc.collect()

In [ ]:
#@title 4. Install dependencies from file in a VirtualEnv
#@markdown Be patient, it takes ~ 5 - 7min <br>
%%capture --no-stderr
#Virtual environment
!pip install pipenv -q
!pip install colab-xterm
%load_ext colabxterm
!pipenv --python 3.8
!pipenv install -r requirements.txt 
gc.collect()


In [ ]:
#@title 5. Mount google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title 6. Drive Path to model
#@markdown Path should start with /content/drive/path-to-your-file <br>
#@markdown <font color="red">Note:</font> Model should be downloaded from https://huggingface.co <br>
#@markdown Lastest release: [Stable-Diffusion-v-1-4](https://huggingface.co/CompVis/stable-diffusion-v-1-4-original)
from os.path import exists

model_path = "/content/drive/MyDrive/AI/models/sd-v1-4.ckpt" #@param {type:"string"}
if exists(model_path)==True:
  print("✅ Valid directory")
else: 
  print("❌ File doesn't exist")

In [ ]:
#@title 7. Symlink to model

from os.path import exists
import os 

# Folder creation if it doesn't exist
if exists("/content/stable-diffusion/models/ldm/stable-diffusion-v1")==True:
  print("❗ Dir stable-diffusion-v1 already exists")
else:
  %mkdir /content/stable-diffusion/models/ldm/stable-diffusion-v1
  print("✅ Dir stable-diffusion-v1 created")

# Symbolic link if it doesn't exist
if exists("/content/stable-diffusion/models/ldm/stable-diffusion-v1/model.ckpt")==True:
  print("❗ Symlink already created")
else: 
  src = model_path
  dst = '/content/stable-diffusion/models/ldm/stable-diffusion-v1/model.ckpt'
  os.symlink(src, dst) 
  print("✅ Symbolic link created successfully")

In [ ]:
#@title 8. Load small ML models required
%%capture --no-stderr
!pip install --ignore-installed Pillow==9.0.0
!pipenv run scripts/preload_models.py
gc.collect()

In [ ]:
#@title 9. Run Terminal and Execute Dream bot
#@markdown <font color="blue">Steps:</font> <br>
#@markdown 1. Execute command `pipenv run scripts/dream.py` to run dream bot.<br>
#@markdown 2. After initialized you'll see `Dream>` line.<br>
#@markdown 3. Example text: `Astronaut floating in a distant galaxy` <br>
#@markdown 4. To quit Dream bot use: `q` command.<br>

#Run from virtual env

%xterm
gc.collect()

In [ ]:
#@title 10. Show generated images

import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

images = []
for img_path in glob.glob('/content/stable-diffusion/outputs/img-samples/*.png'):
    images.append(mpimg.imread(img_path))

# Remove ticks and labels on x-axis and y-axis both

plt.figure(figsize=(20,10))

columns = 5
for i, image in enumerate(images):
    ax = plt.subplot(len(images) / columns + 1, columns, i + 1)
    ax.axes.xaxis.set_visible(False)
    ax.axes.yaxis.set_visible(False)
    ax.axis('off')
    plt.imshow(image)
    gc.collect()



In [ ]:
#@title 11. Fine-tune on an object
%mkdir training_data
!pip install Pillow==9.0.0
!pwd
#@title 12. download 3-5 jpg images to train with
# resize images
from PIL import Image
import glob

size = (512, 512)

# get all the files in a folder, make sure all are image files
files = glob.glob('drive/MyDrive/AI/data/raw/*')
print(len(files))
for fil in files:
    # implement file type checking here if required
    # get the basename, e.g. "dragon.jpg" -> ("dragon", ".jpg")
    basename = os.path.splitext(os.path.basename(fil))[0]
    print("base name: {0}".format(basename))
    with Image.open(fil) as img:
        # resize the image to 512 x 512
        img = img.resize(size)

        # save the resized image, modify the resample method if required, modify the output directory as well
        img.save(f"drive/MyDrive/AI/data/resized/{basename}.png", format="PNG", resample=Image.NEAREST)

print("Resized {0} images".format(len(files)))


In [ ]:
!pip install pytorch_lightning
!pip install omegaconf
!pip install einops
!pip install taming-transformers

In [ ]:
#@title 13. Training
!cd 
!python stable-diffusion/main.py --base stable-diffusion/configs/stable-diffusion/v1-finetune.yaml -t --actual_resume stable-diffusion/models/ldm/stable-diffusion-v1/model.ckpt -n my_key --gpus 0, --data_root /drive/MyDrive/AI/data/resized/